__Subject:__ Topics of Tweets of US Congress through the Lens of Campaign Finance

__Date:__ 09/14/2018

__Author:__ Edmund D. Chitwood

__Summary:__

The following Notebook

- shows the amount of PAC money received by Congress Members by state, party, etc.
- imports and abbreviates topics for analysis,
- shows the top Tweet topics by Congress Members who received the most and least money from PACs to finance their most recent campaigns, and
- explores the high level differences between in PAC contributions depending on what Congress Members Tweet about.

In [238]:
import pandas as pd
import pickle
import numpy as np
from gensim import corpora, models, similarities
import seaborn as sns
sns.set() # need to set to get seaborn defaults

In [170]:
contributions_pacs = pd.read_pickle('contributions_pacs.pkl')

In [171]:
contributions_pacs.head()

,os_id,pacs
0,N00000078,744634
1,N00000153,2019609
2,N00000179,479044
3,N00000270,2513089
4,N00000362,1894962


In [240]:
tweet_topics = pd.read_pickle('tweet_topics_links.pkl')

In [172]:
tweet_topics.head()

,index,tweets,0,1,2,3,4,5,6,7,...,41,42,43,44,45,46,47,48,49,username
0,0,https://twitter.com/AnthonyBrownMD4/status/103...,NaN,NaN,NaN,0.051439,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.049822,NaN,NaN,NaN,anthonybrownmd4
1,1,https://twitter.com/AnthonyBrownMD4/status/103...,NaN,0.0408,NaN,NaN,NaN,NaN,NaN,0.0408,...,0.0408,NaN,NaN,NaN,NaN,0.040800,NaN,NaN,0.0408,anthonybrownmd4
2,2,https://twitter.com/AnthonyBrownMD4/status/999...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anthonybrownmd4
3,3,https://twitter.com/AnthonyBrownMD4/status/985...,NaN,NaN,0.410252,0.050610,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anthonybrownmd4
4,4,https://twitter.com/AnthonyBrownMD4/status/974...,NaN,NaN,NaN,0.231917,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.108892,NaN,NaN,NaN,anthonybrownmd4


In [173]:
# For merging later:
tweet_topics['username'] = (tweet_topics.tweets.apply
                           (lambda x: x.split('/')[3].lower()))

In [174]:
# Drop Rep. Chris Collins
# https://www.opensecrets.org/members-of-congress/summary?cid=N00001285&cycle=2018
# Collins suspended his campaign on August 11, 2018 after he was indicted for
# insider trading. 
(tweet_topics.drop(tweet_topics[tweet_topics
                                .username=='repchriscollins']
                                .index, inplace=True))

In [175]:
congress_metadata = pd.read_pickle('congress_metadata.pkl')

In [176]:
congress_metadata.head()

,last_name,first_name,birthday,gender,type,state,district,senate_class,party,url,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,1952-11-09,M,sen,OH,NaN,1.0,Democrat,https://www.brown.senate.gov,...,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,1958-10-13,F,sen,WA,NaN,1.0,Democrat,https://www.cantwell.senate.gov,...,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,1943-10-05,M,sen,MD,NaN,1.0,Democrat,https://www.cardin.senate.gov,...,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,1947-01-23,M,sen,DE,NaN,1.0,Democrat,https://www.carper.senate.gov/public,...,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,1960-04-13,M,sen,PA,NaN,1.0,Democrat,https://www.casey.senate.gov,...,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.


In [177]:
# Drop Representative Chris Collins.
(congress_metadata.drop(congress_metadata[congress_metadata.opensecrets_id
                                          =='N00001285'].index, inplace=True))


In [178]:
tweet_topics.head()

,index,tweets,0,1,2,3,4,5,6,7,...,41,42,43,44,45,46,47,48,49,username
0,0,https://twitter.com/AnthonyBrownMD4/status/103...,NaN,NaN,NaN,0.051439,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.049822,NaN,NaN,NaN,anthonybrownmd4
1,1,https://twitter.com/AnthonyBrownMD4/status/103...,NaN,0.0408,NaN,NaN,NaN,NaN,NaN,0.0408,...,0.0408,NaN,NaN,NaN,NaN,0.040800,NaN,NaN,0.0408,anthonybrownmd4
2,2,https://twitter.com/AnthonyBrownMD4/status/999...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anthonybrownmd4
3,3,https://twitter.com/AnthonyBrownMD4/status/985...,NaN,NaN,0.410252,0.050610,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anthonybrownmd4
4,4,https://twitter.com/AnthonyBrownMD4/status/974...,NaN,NaN,NaN,0.231917,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.108892,NaN,NaN,NaN,anthonybrownmd4


In [179]:
contributions_meta = (pd.merge(contributions_pacs, congress_metadata, left_on
                              ='os_id', right_on='opensecrets_id'))

In [180]:
contributions_meta.columns

Index(['os_id', 'pacs', 'last_name', 'first_name', 'birthday', 'gender',
       'type', 'state', 'district', 'senate_class', 'party', 'url', 'address',
       'phone', 'contact_form', 'rss_url', 'twitter', 'facebook', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'fec_ids', 'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id'],
      dtype='object')

In [181]:
contributions_meta = (contributions_meta[['os_id','pacs','state','party',
                                          'twitter', 'last_name','first_name',
                                          'gender','senate_class','type']])

In [237]:
# The majority of Congress Members received under $1 million from PACs.
# However, several Congress Members received over $5 million from PACs.
contributions_meta.pacs.hist(bins=10)

In [184]:
# The average Congress Member took almost $1,000,000 dollars from PACS to 
# to finance their most recent campaign.
(contributions_meta
    .pacs
    .mean())

997719.6213017751

In [186]:
# The average Republican Congress Member took $1.1 Million dollars, over
# $200,000 more than the average Democrat took.
(contributions_meta
    .groupby('party')
    .pacs
    .mean())

party
Democrat       8.785913e+05
Independent    6.627935e+05
Republican     1.101411e+06
Name: pacs, dtype: float64

In [187]:
# Male Members of Congress took over $150,000 more than did Female Members of 
# Congress.
(contributions_meta
    .groupby('gender')
    .pacs
    .mean())

gender
F    8.619670e+05
M    1.033177e+06
Name: pacs, dtype: float64

In [19]:
# The five states whose Congress Members received the most PAC money 
# during their last campaigns have just one Member in the House of 
# Representatives, due to their low populations. Therefore, the ratios of 
# US Senators to US Representatives in those states are uncommonly high.
(contributions_meta
    .groupby('state')
    .pacs
    .mean()
    .sort_values(ascending=False)
    .head())

state
MT    2.801525e+06
DE    2.079046e+06
ND    2.000293e+06
SD    1.937467e+06
WY    1.898909e+06
Name: pacs, dtype: float64

In [20]:
# US Senators receive about four times as much PAC money as US
# Representatives do.
(contributions_meta
    .groupby('type')
    .pacs
    .mean())

type
rep    6.432884e+05
sen    2.515128e+06
Name: pacs, dtype: float64

In [194]:
# The states whose US Senators took the most money from PACs on
# average:
(contributions_meta[contributions_meta.type=='sen']
    .groupby(['state'])
    .pacs
    .mean()
    .sort_values(ascending=False)
    .head())

state
OH    4585344.0
PA    4448134.5
SC    3936951.5
MO    3798196.5
NC    3513402.5
Name: pacs, dtype: float64

In [196]:
# The states whose US Senators took the least money from PACs on 
# average:
(contributions_meta[contributions_meta.type=='sen']
    .groupby(['state'])
    .pacs
    .mean()
    .sort_values(ascending=True)
    .head())

state
VT     841816.5
CA    1086992.5
AZ    1461868.0
MA    1475381.0
RI    1573027.5
Name: pacs, dtype: float64

In [193]:
# The states whose US Representatives took the most money from PACs on 
# average:
(contributions_meta[contributions_meta.type=='rep']
    .groupby(['state'])
    .pacs
    .mean()
    .sort_values(ascending=False)
    .head())

state
OR    1.129690e+06
MO    9.865974e+05
LA    9.486040e+05
IL    8.949859e+05
WI    8.287786e+05
Name: pacs, dtype: float64

In [192]:
# The states whose US Representatives took the least money from PACs on 
# average: (Puerto Rico and American Samoa are included)
(contributions_meta[contributions_meta.type=='rep']
    .groupby(['state'])
    .pacs
    .mean()
    .sort_values(ascending=True)
    .head())

state
NM    17500.0
SD    24225.0
PR    26700.0
AS    40250.0
HI    72491.5
Name: pacs, dtype: float64

In [197]:
contributions_meta_topics = (pd.merge(contributions_meta,
                                     tweet_topics, left_on='twitter',
                                     right_on='username'))

In [198]:
contributions_meta_topics.head()

,os_id,pacs,state,party,twitter,last_name,first_name,gender,senate_class,type,...,41,42,43,44,45,46,47,48,49,username
0,N00000078,744634,NY,Democrat,repmaloney,Maloney,Carolyn,F,NaN,rep,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,repmaloney
1,N00000078,744634,NY,Democrat,repmaloney,Maloney,Carolyn,F,NaN,rep,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,repmaloney
2,N00000078,744634,NY,Democrat,repmaloney,Maloney,Carolyn,F,NaN,rep,...,NaN,NaN,NaN,NaN,NaN,0.102,NaN,NaN,NaN,repmaloney
3,N00000078,744634,NY,Democrat,repmaloney,Maloney,Carolyn,F,NaN,rep,...,NaN,NaN,NaN,NaN,NaN,0.102,NaN,NaN,NaN,repmaloney
4,N00000078,744634,NY,Democrat,repmaloney,Maloney,Carolyn,F,NaN,rep,...,NaN,NaN,NaN,NaN,NaN,0.051,NaN,NaN,NaN,repmaloney


In [199]:
contributions_meta_topics.columns

Index([       'os_id',         'pacs',        'state',        'party',
            'twitter',    'last_name',   'first_name',       'gender',
       'senate_class',         'type',        'index',       'tweets',
                    0,              1,              2,              3,
                    4,              5,              6,              7,
                    8,              9,             10,             11,
                   12,             13,             14,             15,
                   16,             17,             18,             19,
                   20,             21,             22,             23,
                   24,             25,             26,             27,
                   28,             29,             30,             31,
                   32,             33,             34,             35,
                   36,             37,             38,             39,
                   40,             41,             42,             43,
      

In [27]:
# Import model to review and abbreviate topics.
lda =  models.LdaModel.load('lda.model')

In [15]:
lda.print_topics(num_topics=50)

2018-09-13 11:33:27,301 : INFO : topic #0 (0.020): 0.079*"va" + 0.061*"take" + 0.046*"federal" + 0.037*"veterans" + 0.035*"better" + 0.031*"government" + 0.027*"ensure" + 0.027*"deserve" + 0.025*"benefits" + 0.025*"speak"
2018-09-13 11:33:27,306 : INFO : topic #1 (0.020): 0.073*"working" + 0.053*"community" + 0.038*"hear" + 0.036*"look" + 0.034*"rights" + 0.032*"children" + 0.028*"kids" + 0.021*"de" + 0.021*"income" + 0.015*"human"
2018-09-13 11:33:27,309 : INFO : topic #2 (0.020): 0.059*"security" + 0.043*"vote" + 0.037*"lets" + 0.034*"access" + 0.032*"republicans" + 0.026*"find" + 0.026*"billion" + 0.025*"obamacare" + 0.025*"social" + 0.022*"repeal"
2018-09-13 11:33:27,313 : INFO : topic #3 (0.020): 0.073*"first" + 0.072*"stop" + 0.037*"oped" + 0.031*"use" + 0.029*"may" + 0.027*"hold" + 0.025*"reduce" + 0.016*"nothing" + 0.016*"old" + 0.016*"officials"
2018-09-13 11:33:27,316 : INFO : topic #4 (0.020): 0.065*"visit" + 0.062*"joined" + 0.041*"colleagues" + 0.027*"weekend" + 0.026*"two

2018-09-13 11:33:27,446 : INFO : topic #38 (0.020): 0.078*"week" + 0.077*"staff" + 0.050*"leaders" + 0.048*"next" + 0.037*"local" + 0.029*"taking" + 0.024*"joe" + 0.022*"stopping" + 0.022*"air" + 0.019*"womens"
2018-09-13 11:33:27,450 : INFO : topic #39 (0.020): 0.093*"tax" + 0.044*"reform" + 0.044*"end" + 0.038*"stand" + 0.035*"cuts" + 0.034*"cut" + 0.027*"class" + 0.026*"middle" + 0.026*"families" + 0.021*"long"
2018-09-13 11:33:27,453 : INFO : topic #40 (0.020): 0.097*"like" + 0.065*"big" + 0.038*"congratulations" + 0.031*"oil" + 0.030*"win" + 0.025*"recovery" + 0.025*"place" + 0.022*"effort" + 0.020*"john" + 0.018*"training"
2018-09-13 11:33:27,455 : INFO : topic #41 (0.020): 0.038*"system" + 0.037*"free" + 0.032*"campaign" + 0.030*"lives" + 0.029*"critical" + 0.029*"decision" + 0.028*"improve" + 0.027*"letter" + 0.020*"urging" + 0.018*"consumers"
2018-09-13 11:33:27,460 : INFO : topic #42 (0.020): 0.080*"year" + 0.064*"family" + 0.030*"thoughts" + 0.029*"friends" + 0.028*"victims"

[(0,
  '0.079*"va" + 0.061*"take" + 0.046*"federal" + 0.037*"veterans" + 0.035*"better" + 0.031*"government" + 0.027*"ensure" + 0.027*"deserve" + 0.025*"benefits" + 0.025*"speak"'),
 (1,
  '0.073*"working" + 0.053*"community" + 0.038*"hear" + 0.036*"look" + 0.034*"rights" + 0.032*"children" + 0.028*"kids" + 0.021*"de" + 0.021*"income" + 0.015*"human"'),
 (2,
  '0.059*"security" + 0.043*"vote" + 0.037*"lets" + 0.034*"access" + 0.032*"republicans" + 0.026*"find" + 0.026*"billion" + 0.025*"obamacare" + 0.025*"social" + 0.022*"repeal"'),
 (3,
  '0.073*"first" + 0.072*"stop" + 0.037*"oped" + 0.031*"use" + 0.029*"may" + 0.027*"hold" + 0.025*"reduce" + 0.016*"nothing" + 0.016*"old" + 0.016*"officials"'),
 (4,
  '0.065*"visit" + 0.062*"joined" + 0.041*"colleagues" + 0.027*"weekend" + 0.026*"two" + 0.023*"weeks" + 0.020*"strengthen" + 0.018*"today" + 0.017*"opening" + 0.016*"park"'),
 (5,
  '0.140*"read" + 0.083*"rt" + 0.032*"mt" + 0.027*"food" + 0.022*"sexual" + 0.020*"street" + 0.020*"assault

In [201]:
# Rename topics numbers with topic interpretations for purposes of Analysis.
contributions_meta_topics.columns = ('os_id','pacs','state','party','twitter',\
                                     'last_name','first_name','gender',\
                                     'senate_class','type','index','tweets',\
                                     'Veteran_Benefits_0','Human_Rights_1',\
                                     'Republicans_Repealing_Obamacare_2',\
                                     'Stops_on_Campaign_Trail_3',
                                     'Visiting_Places_4','Sexual_Assault_5',\
                                     'Supreme_Court_Appointments_6',\
                                     'Free_Trade_7','Crises_and_Conflicts_8',\
                                     'Time_for_Change_9',\
                                     'Special_Investigation_10',\
                                     'Wall_Street_11',\
                                     'Sign_Law_12','Make_Voice_Heard_13',\
                                     'Getting_Things_Done_14',\
                                     'Defense_Secretary_15','Legislation_16',\
                                     'Energy_Deal_17','Congress_18',\
                                     'Support_the_Troops_19','President_20',\
                                     'Happy_Birthday_America_21',\
                                     'US_Economy_and_Jobs_22','Facebook_23',\
                                     'Positivity_and_Learning_24',\
                                     'Bernie_Sanders_Campaign_25',\
                                     'Hearings_26','Meetings_27',\
                                     'Togetherness_28',
                                     'Issuing_Statements_29',\
                                     'Government_Services_30',\
                                     'Young_People_in_Politics_31',\
                                     'Financial_Issues_32','Health_Care_33',\
                                     'Education_34','Members_of_Congress_35',\
                                     'Videos_of_Tours_36','Looking_Forward_37',\
                                     'Congressional_Staff_38','Tax_Reform_39',\
                                     'Congratulations_40',
                                     'Political_System__41',\
                                     'Thoughts_and_Prayers_to_Family_and_Friends_of_Victims_42',\
                                     'Small_Business_43',
                                     'Education_Policy_44','The_Deficit_45',\
                                     'One_Know_Could_46', 'The_Public_47',\
                                      'Senate_48','Honors_49','username')

In [244]:
# Function prints out the amount a Member of Congress Tweets about each topic,
#(given their Opensecrets ID). Amount of topic is averageg topic representation 
# multiplied by the number of times topic appears in the the Congress Member's
# Tweets.

def cm_topics(id_num):
    
    topic_counts = (contributions_meta_topics[contributions_meta_topics.os_id
                    ==id_num][['Veteran_Benefits_0','Human_Rights_1',
                               'Republicans_Repealing_Obamacare_2',
                               'Stops_on_Campaign_Trail_3',
                               'Visiting_Places_4','Sexual_Assault_5',
                               'Supreme_Court_Appointments_6','Free_Trade_7',
                               'Crises_and_Conflicts_8','Time_for_Change_9',
                               'Special_Investigation_10','Wall_Street_11',
                               'Sign_Law_12','Make_Voice_Heard_13',
                               'Getting_Things_Done_14',
                               'Defense_Secretary_15','Legislation_16',
                               'Energy_Deal_17','Congress_18',
                               'Support_the_Troops_19','President_20',
                               'Happy_Birthday_America_21',
                               'US_Economy_and_Jobs_22','Facebook_23',
                               'Positivity_and_Learning_24',
                               'Bernie_Sanders_Campaign_25','Hearings_26',
                               'Meetings_27','Togetherness_28',
                               'Issuing_Statements_29',
                               'Government_Services_30',
                               'Young_People_in_Politics_31',
                               'Financial_Issues_32','Health_Care_33',
                               'Education_34','Members_of_Congress_35',
                               'Videos_of_Tours_36','Looking_Forward_37',
                               'Congressional_Staff_38','Tax_Reform_39',
                               'Congratulations_40','Political_System__41',
                               'Thoughts_and_Prayers_to_Family_and_Friends_of_Victims_42',
                               'Small_Business_43','Education_Policy_44',
                               'The_Deficit_45','One_Know_Could_46', 
                               'The_Public_47','Senate_48','Honors_49']]
                               .count()[:])

    topic_means = (contributions_meta_topics[contributions_meta_topics.os_id
                    ==id_num][['Veteran_Benefits_0','Human_Rights_1',
                               'Republicans_Repealing_Obamacare_2',
                               'Stops_on_Campaign_Trail_3',
                               'Visiting_Places_4','Sexual_Assault_5',
                               'Supreme_Court_Appointments_6','Free_Trade_7',
                               'Crises_and_Conflicts_8','Time_for_Change_9',
                               'Special_Investigation_10','Wall_Street_11',
                               'Sign_Law_12','Make_Voice_Heard_13',
                               'Getting_Things_Done_14',
                               'Defense_Secretary_15','Legislation_16',
                               'Energy_Deal_17','Congress_18',
                               'Support_the_Troops_19','President_20',
                               'Happy_Birthday_America_21',
                               'US_Economy_and_Jobs_22','Facebook_23',
                               'Positivity_and_Learning_24',
                               'Bernie_Sanders_Campaign_25','Hearings_26',
                               'Meetings_27','Togetherness_28',
                               'Issuing_Statements_29',
                               'Government_Services_30',
                               'Young_People_in_Politics_31',
                               'Financial_Issues_32','Health_Care_33',
                               'Education_34','Members_of_Congress_35',
                               'Videos_of_Tours_36','Looking_Forward_37',
                               'Congressional_Staff_38','Tax_Reform_39',
                               'Congratulations_40','Political_System__41',
                               'Thoughts_and_Prayers_to_Family_and_Friends_of_Victims_42',
                               'Small_Business_43','Education_Policy_44',
                               'The_Deficit_45','One_Know_Could_46', 
                               'The_Public_47','Senate_48','Honors_49']]
                               .mean()[:])

    return topic_means * topic_counts

In [207]:
# The the five Members of Congress who received the most money from PACs
# each received over $5,000,000 dollars.
top5 = (contributions_meta_topics
        .groupby('os_id')
        .pacs
        .mean()
        .sort_values(ascending=False)
        .head())

top5

os_id
N00003682    5879797
N00031782    5775093
N00001489    5520915
N00005195    5397110
N00003389    5232677
Name: pacs, dtype: int64

In [246]:
for i in top5.index:
    (print(congress_metadata[congress_metadata.opensecrets_id==i]
          [['first_name','last_name','type','state','party']]))

    first_name last_name type state       party
221     Robert   Portman  sen    OH  Republican
    first_name last_name type state       party
255        Tim     Scott  sen    SC  Republican
    first_name last_name type state       party
280    Patrick    Toomey  sen    PA  Republican
   first_name last_name type state       party
49        Roy     Blunt  sen    MO  Republican
   first_name  last_name type state       party
25      Mitch  McConnell  sen    KY  Republican


In [245]:
#Rob Portman's top topics:
cm_topics('N00003682').sort_values(ascending=False).head()

Meetings_27              293.633300
Support_the_Troops_19    263.583560
Honors_49                246.247045
Congress_18              246.074930
Legislation_16           227.286348
dtype: float64

In [71]:
# Tim Scott's top topics:
cm_topics('N00031782').sort_values(ascending=False).head()

Meetings_27                    121.663895
Honors_49                       89.872520
Hearings_26                     81.974922
Young_People_in_Politics_31     69.992718
Facebook_23                     64.755770
dtype: float64

In [72]:
# Patrick Toomey's top topics:
cm_topics('N00001489').sort_values(ascending=False).head()

Meetings_27               69.162666
Honors_49                 52.125286
Senate_48                 49.326346
Members_of_Congress_35    48.069722
Hearings_26               46.722641
dtype: float64

In [209]:
# The the five Members of Congress who received the least from PACs:
bottom5 = (contributions_meta_topics
           .groupby('os_id')
           .pacs
           .mean()
           .sort_values(ascending=True)
           .head())

bottom5

os_id
N00029127   -10500
N00040007    -4159
N00027751    -1901
N00008028        0
N00029077     6500
Name: pacs, dtype: int64

In [212]:
for i in bottom5.index:
    (print(congress_metadata[congress_metadata.opensecrets_id==i]
          [['first_name','last_name','type','state','party']]))

    first_name last_name type state     party
220      Jared     Polis  rep    CO  Democrat
    first_name last_name type state       party
477    Francis    Rooney  rep    FL  Republican
    first_name last_name type state     party
245       John  Sarbanes  rep    MD  Democrat
    first_name last_name type state       party
151        Sam   Johnson  rep    TX  Republican
    first_name last_name type state       party
146       Lynn   Jenkins  rep    KS  Republican


In [73]:
# Jared Polis' top topics:
cm_topics('N00029127').sort_values(ascending=False).head()

Meetings_27              27.589816
Legislation_16           20.262529
Support_the_Troops_19    16.928278
Health_Care_33           14.817569
Congress_18              14.369678
dtype: float64

In [74]:
# Francis Rooney's top topics:
cm_topics('N00040007').sort_values(ascending=False).head()

Honors_49                 5.959103
Hearings_26               5.827413
US_Economy_and_Jobs_22    4.056930
Meetings_27               3.891491
Legislation_16            3.539622
dtype: float64

In [76]:
# John Sarbanes' top topics:
cm_topics('N00027751').sort_values(ascending=False).head()

Young_People_in_Politics_31    15.206357
Special_Investigation_10       14.116563
Visiting_Places_4               8.929187
Health_Care_33                  8.718143
Meetings_27                     7.702872
dtype: float64

In [213]:
# Topics with 'political substance':
# Topics about which on which there are common differences  opinion along 
# politically partisan lines.
substance = contributions_meta_topics[['Veteran_Benefits_0','Human_Rights_1',
                                       'Republicans_Repealing_Obamacare_2',
                                       'Sexual_Assault_5',
                                       'Supreme_Court_Appointments_6',
                                       'Free_Trade_7',
                                       'Crises_and_Conflicts_8',
                                       'Special_Investigation_10',
                                       'Wall_Street_11','Energy_Deal_17',
                                       'US_Economy_and_Jobs_22','Facebook_23',
                                       'Bernie_Sanders_Campaign_25',
                                       'Government_Services_30',
                                       'Financial_Issues_32','Health_Care_33',
                                       'Education_34','Tax_Reform_39',
                                       'Political_System__41',
                                       'Small_Business_43',
                                       'Education_Policy_44',
                                       'The_Deficit_45',]]

In [164]:
# Topics without 'political substance':
no_substance = (contributions_meta_topics[['Stops_on_Campaign_Trail_3',
                                           'Visiting_Places_4',
                                           'Time_for_Change_9',
                                           'Sign_Law_12',
                                           'Make_Voice_Heard_13',
                                           'Getting_Things_Done_14',
                                           'Defense_Secretary_15',
                                           'Legislation_16',
                                           'Congress_18',
                                           'Support_the_Troops_19',
                                           'President_20',
                                           'Happy_Birthday_America_21',
                                           'Positivity_and_Learning_24',
                                           'Hearings_26','Meetings_27',
                                           'Togetherness_28',
                                           'Issuing_Statements_29',
                                           'Young_People_in_Politics_31',
                                           'Members_of_Congress_35',
                                           'Videos_of_Tours_36',
                                           'Looking_Forward_37',
                                           'Congressional_Staff_38',
                                           'Congratulations_40',
                                           'Thoughts_and_Prayers_to_Family_and_Friends_of_Victims_42',
                                           'One_Know_Could_46', 'The_Public_47',
                                           'Senate_48','Honors_49']])

In [231]:
# Iterate through Tweet topics DataFrame and determine which Tweets are
# relatively politically substantial and those which are not.
substantial = []
not_substantial = []

for index, row in contributions_meta_topics.iterrows():
    if substance.loc[index].sum(axis=0) > no_substance.loc[index].sum(axis=0):
        substantial.append(index)
    elif substance.loc[index].sum(axis=0) < no_substance.loc[index].sum(axis=0):
        not_substantial.append(index)

In [227]:
# Get the amount of PAC money received by the authors of substantial Tweets,
# and print the average amount. Authors of substantial Tweets receive on average 
# $1.3 million dollars from PACs.
pac_money = []

for i in substantial:
    pac_money.append(contributions_meta_topics.loc[i].pacs)

np.asarray(pac_money).mean()

1337212.408401821

In [229]:
# Get the amount of PAC money received by the authors of insubstantial Tweets,
# and print the average amount. Authors of insubstantial Tweets receive on average 
# $1.4 million dollars from PACs.
pac_money = []

for i in not_substantial:
    pac_money.append(contributions_meta_topics.loc[i].pacs)

np.asarray(pac_money).mean()

1417342.2675836545

In [ ]:
# Both averages are substantially higher than the average amount of PAC
# money received by all candidates, presumably indicating that those
# Congress Members who Tweet more also receive more money from PACS.